In [45]:
import os
import requests
import json
import urllib3
import pandas as pd
import jupytab

In [2]:
http_proxy  = os.environ.get('HTTP_PROXY')
https_proxy = os.environ.get('HTTPS_PROXY')

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
proxies = {}

if http_proxy:
    proxies["http"] = http_proxy
    print("HTTP proxy defined as {}".format(http_proxy))
else:
    print("No HTTP proxy defined")
    
if https_proxy:
    proxies["https"] = https_proxy
    print("HTTPS proxy defined as {}".format(http_proxy))
else:
    print("No HTTPS proxy defined")


No HTTP proxy defined
No HTTPS proxy defined


In [38]:
def covid_refresh():
    try:
        states = requests.get(url='https://api.covid19india.org/districts_daily.json',
                                      proxies=proxies,
                                      verify=False)
        states=states.json()["districtsDaily"]
        
        
    except Exception as e:
        states_json = [{'icao24': "Khan error -"+str(e)}]
    
    df=pd.DataFrame(columns=['State','district','active','confirmed','deceased','recovered','date'])
    
    for (k1,v1) in states.items():
            district=v1
            for (k2,v2) in district.items():
                for val in v2:
                    df=df.append({'State':k1,'district':k2,'active':int(val['active']),'confirmed':int(val['confirmed'])
                               ,'deceased':int(val['deceased']),'recovered':int(val['recovered']),'date':val['date']},ignore_index=True)
    
    return df

    

In [39]:
covid_19_df = covid_refresh()

In [42]:
tables=jupytab.Tables()
tables['covid_19_india']=jupytab.DataFrameTable('covid_19_india', dataframe=covid_19_df, refresh_method=covid_refresh)

In [43]:
# GET /schema
tables.render_schema()

[{"id": "covid_19_india", "alias": "covid_19_india", "columns": [{"id": "State", "dataType": "string"}, {"id": "district", "dataType": "string"}, {"id": "active", "dataType": "string"}, {"id": "confirmed", "dataType": "string"}, {"id": "deceased", "dataType": "string"}, {"id": "recovered", "dataType": "string"}, {"id": "date", "dataType": "string"}]}]


In [44]:
# GET /data
tables.render_data(REQUEST)

NameError: name 'REQUEST' is not defined